In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

%matplotlib inline

In [2]:
boston = load_boston()

In [3]:
df = pd.DataFrame(boston.data,columns=boston.feature_names)

df['PRICE'] = boston.target
print(df.shape)

(506, 14)


In [4]:
y_target = df['PRICE']
x_data = df.drop(['PRICE'],axis=1,inplace=False)

X_train,X_test,y_train,y_test = train_test_split(x_data,y_target,test_size=0.3,random_state=156)

In [5]:
lr = LinearRegression()
lr.fit(X_train,y_train)
 
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test,y_preds)
rmse = np.sqrt(mse)

print("MSE :",mse)
print("RMSE : ",rmse)
print("R2 : ",r2_score(y_test,y_preds))

MSE : 17.29691590790215
RMSE :  4.158956107955715
R2 :  0.7572263323138921


In [6]:
print("절편 값 : ",lr.intercept_)
print("회귀계수 : ",np.round(lr.coef_,1))

절편 값 :  40.995595172164336
회귀계수 :  [ -0.1   0.1   0.    3.  -19.8   3.4   0.   -1.7   0.4  -0.   -0.9   0.
  -0.6]


In [7]:
lr.coef_

array([-1.12979614e-01,  6.55124002e-02,  3.44366694e-02,  3.04589777e+00,
       -1.97958320e+01,  3.35496880e+00,  5.93713290e-03, -1.74185354e+00,
        3.55884364e-01, -1.42954516e-02, -9.20180066e-01,  1.03966156e-02,
       -5.66182106e-01])

In [8]:
# 회귀계수 큰 값 순으로 정렬
coeff = pd.Series(data=np.round(lr.coef_,1),index=x_data.columns)
coeff.sort_values(ascending=False)

RM          3.4
CHAS        3.0
RAD         0.4
ZN          0.1
B           0.0
TAX        -0.0
AGE         0.0
INDUS       0.0
CRIM       -0.1
LSTAT      -0.6
PTRATIO    -0.9
DIS        -1.7
NOX       -19.8
dtype: float64

In [9]:
# cross_val_score

from sklearn.model_selection import cross_val_score

neg_mse_scores = cross_val_score(lr,x_data,y_target,scoring="neg_mean_squared_error",cv=5)
rmse_scores = np.sqrt(-1*neg_mse_scores)
avg_rmse = np.mean(rmse_scores)

# coross_val_score(scoring="neg_mean_squared_error")로 반환된 값은 모두 음수
print("5 folds 의 개별 Negative MSE scores :",np.round(neg_mse_scores,2))
print("5 folds 의 개별 RMSE scores :",np.round(rmse_scores,2))
print("5 folds 의 평균 RMSE :",avg_rmse)

5 folds 의 개별 Negative MSE scores : [-12.46 -26.05 -33.07 -80.76 -33.31]
5 folds 의 개별 RMSE scores : [3.53 5.1  5.75 8.99 5.77]
5 folds 의 평균 RMSE : 5.828658946215808


## 선형 회귀 모델을 위한 데이터 변환

In [29]:
from sklearn.linear_model import Lasso,ElasticNet,Ridge

# alpha값에 따른 회귀 모델의 폴드 평균 RMSE를 출력하고 회귀 계수값들을 DataFrame으로 반환
def get_linear_reg_eval(model_name,params=None,X_data_n=None,y_target_n=None,verbose=True):
    coeff_df = pd.DataFrame()
    
    if verbose : print("####",model_name,"####")
    
    for param in params:
        if model_name == "Ridge" : model = Ridge(alpha=param)
        elif model_name == "Lasso" : model = Lasso(alpha=param)            
        elif model_name == "ElasticNet" : model = ElasticNet(alpha=param,l1_ratio=0.7)            
        
        neg_mse_scores = cross_val_score(model,X_data_n,y_target_n,scoring='neg_mean_squared_error',cv=5)
        avg_rmse = np.mean(np.sqrt(-1*neg_mse_scores))
        print('alpha {0}일때 5 폴드 세트의 평균 RMSE : {1:3f}'.format(param,avg_rmse))
        # cross_val_score 는 evaluation metric만 반환하므로 모델을 다시 학습하여 회귀 계수 추출
        model.fit(x_data,y_target)
        # alpha에 따른 피처별 회귀 계수를 Series로 변환하고 이를 DataFrame의 칼럼으로 추가
        coeff = pd.Series(data=model.coef_,index=x_data.columns)
        colname='alpha:'+str(param)
        coeff_df[colname] = coeff
    return coeff_df

In [35]:
# 표준정규분포,최대최소,로그변환 , 다항식

from sklearn.preprocessing import StandardScaler,MinMaxScaler,PolynomialFeatures

def get_scaled_data(method='None',p_degree=None, input_data=None):
    if method == "Standard":
        scaled_data = StandardScaler().fit_transform(input_data)
    elif method == "MinMax":
        scaled_data = MinMaxScaler().fit_transform(input_data)
    elif method == "Log":
        scaled_data = np.log1p(input_data)
    else :
        scaled_data = input_data
    
    if p_degree != None:
        scaled_data = PolynomialFeatures(degree=p_degree, include_bias=False).fit_transform(scaled_data)
    
    return scaled_data

In [36]:
# Ridge의 alpha값을 다르게 적용하고 다양한 데이터 변환 방법에 따른 RMSE 추출 
alphas = [0.1,1,10,100]

scale_methods = [(None,None),('Standard',None),('Standard',2),('MinMax',None),('MinMax',2),('Log',None)]

for scale_method in scale_methods:
    X_data_scaled = get_scaled_data(method=scale_method[0],p_degree=scale_method[1],input_data=x_data)
    
    print('\n## 변환 유형:{0}, Polynomial Degree:{1}'.format(scale_method[0],scale_method[1]))
    get_linear_reg_eval('Ridge',params=alphas,X_data_n=X_data_scaled,y_target_n=y_target,verbose=False)


## 변환 유형:None, Polynomial Degree:None
alpha 0.1일때 5 폴드 세트의 평균 RMSE : 5.788487
alpha 1일때 5 폴드 세트의 평균 RMSE : 5.652571
alpha 10일때 5 폴드 세트의 평균 RMSE : 5.518166
alpha 100일때 5 폴드 세트의 평균 RMSE : 5.329590

## 변환 유형:Standard, Polynomial Degree:None
alpha 0.1일때 5 폴드 세트의 평균 RMSE : 5.825993
alpha 1일때 5 폴드 세트의 평균 RMSE : 5.802890
alpha 10일때 5 폴드 세트의 평균 RMSE : 5.636831
alpha 100일때 5 폴드 세트의 평균 RMSE : 5.421375

## 변환 유형:Standard, Polynomial Degree:2
alpha 0.1일때 5 폴드 세트의 평균 RMSE : 8.827236
alpha 1일때 5 폴드 세트의 평균 RMSE : 6.871288
alpha 10일때 5 폴드 세트의 평균 RMSE : 5.484870
alpha 100일때 5 폴드 세트의 평균 RMSE : 4.634438

## 변환 유형:MinMax, Polynomial Degree:None
alpha 0.1일때 5 폴드 세트의 평균 RMSE : 5.763570
alpha 1일때 5 폴드 세트의 평균 RMSE : 5.465045
alpha 10일때 5 폴드 세트의 평균 RMSE : 5.754164
alpha 100일때 5 폴드 세트의 평균 RMSE : 7.634920

## 변환 유형:MinMax, Polynomial Degree:2
alpha 0.1일때 5 폴드 세트의 평균 RMSE : 5.297560
alpha 1일때 5 폴드 세트의 평균 RMSE : 4.322660
alpha 10일때 5 폴드 세트의 평균 RMSE : 5.185204
alpha 100일때 5 폴드 세트의 평균 RMSE : 6.537867

## 변환 유형:Log,